In [2]:
import pandas as pd

In [7]:
from google2pandas import *
conn = GoogleAnalyticsQuery(secrets='../auth/client_secret.json', token_file_name="../auth/")
 

SystemExit: The client secrets were invalid: 
('Error opening file', '../auth/client_secret.json', 'No such file or directory', 2)
WARNING: Please configure OAuth 2.0

To make this sample run you will need to populate the client_secrets.json file
found at:

   ../auth/client_secret.json

with information from the APIs Console <https://code.google.com/apis/console>.



/Users/Lee/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
conn.execute_query

In [1]:
def get_ga_data(eventName,startDate, endDate, preStartDate, preEndDate):

    conn = GoogleAnalyticsQuery(secrets='client_secret.json', token_file_name='analytics.dat')
    metrics = ['sessions','transactions','transactionsPerSession','revenuePerTransaction','transactionRevenue']
    
    segments = [{'syntax':'sessions::condition::!ga:eventAction=@Banner','name':'No Banner Click'},
                {'syntax':'sessions::condition::ga:eventAction=@Banner','name':'Banner Click'}]
    
    df = pd.DataFrame()
    
    for segment in segments:
        query = {
        'ids' : '91998326',
        'metrics' : metrics,
        'start_date' : startDate,
        'end_date': endDate,
        'segment': segment['syntax'],
        }
       
        df_holder, metadata = conn.execute_query(**query)
        df_holder['segment'] = segment['name']
        df_holder['Period Start'] = startDate
        df_holder['Period End'] = endDate
        
        query = {
        'ids' : '91998326',
        'metrics' : metrics,
        'start_date' : preStartDate,
        'end_date': preEndDate,
        'segment': segment['syntax'],
        }

        pre_df_holder, metadata = conn.execute_query(**query)
        pre_df_holder.columns = ['pre-sessions','pre-transactions','pre-transactionsPerSession','pre-revenuePerTransaction','pre-transactionRevenue']
        pre_df_holder['Pre Period Start'] = preStartDate
        pre_df_holder['Pre Period End'] = preEndDate
        df_loop_holder = pd.concat([df_holder,pre_df_holder], axis=1, join='inner')
        
        df = df.append(df_loop_holder)
    
    query = {
    'ids' : '91998326',
    'metrics' : metrics,
    'start_date' : startDate,
    'end_date': endDate,
    }

    df_holder, metadata = conn.execute_query(**query)
    df_holder['segment'] = 'All Users'
    df_holder['Period Start'] = startDate
    df_holder['Period End'] = endDate
    
    query = {
    'ids' : '91998326',
    'metrics' : metrics,
    'start_date' : preStartDate,
    'end_date': preEndDate,
    }

    pre_df_holder, metadata = conn.execute_query(**query)
    pre_df_holder.columns = ['pre-sessions','pre-transactions','pre-transactionsPerSession','pre-revenuePerTransaction','pre-transactionRevenue']
    pre_df_holder['Pre Period Start'] = preStartDate
    pre_df_holder['Pre Period End'] = preEndDate
    
    df_loop_holder = pd.concat([df_holder,pre_df_holder], axis=1, join='inner')

    df = df.append(df_loop_holder)
        
        
    df['event name'] = eventName
    df['transactionRevenue'] = df['transactionRevenue'].astype(float)
    df['transactionRevenue'] = df['transactionRevenue'].astype(str)
    df['pre-transactionRevenue'] = df['pre-transactionRevenue'].astype(float)
    df['pre-transactionRevenue'] = df['pre-transactionRevenue'].astype(str)

    return df

In [4]:
df = get_ga_data('May','2017-01-01', '2017-02-01','2016-01-01','2016-02-01' )
df = df.append(get_ga_data('May','2017-01-01', '2017-02-01','2016-01-01','2016-02-01' ))


In [5]:
df = get_ga_data('Black Friday 2016 - Pre Event' , '2016-11-18' , '2016-11-24' , '2015-01-01' , '2015-01-01')
df = df.append(get_ga_data('Black Friday 2016 - BF Weekend' , '2016-11-24' , '2016-11-28' , '2015-01-01' , '2015-01-01'))
df = df.append(get_ga_data('Bank Holiday Spring YoY - Easter Pre Event' , '2017-04-06' , '2017-04-13' , '2016-04-21' , '2016-04-21'))
df = df.append(get_ga_data('Bank Holiday Spring YoY - Weekend Event' , '2017-04-13' , '2017-04-19' , '2016-04-19' , '2016-04-30'))
df = df.append(get_ga_data('Bank Holiday Spring YoY - Monday 1 day event' , '2017-04-17' , '2017-04-17' , '2016-04-28' , '2016-04-28'))
df = df.append(get_ga_data('Early May - Pre Event' , '2017-04-20' , '2017-04-26' , '2016-04-22' , '2016-04-29'))
df = df.append(get_ga_data('Early May - Weekend Event' , '2017-04-26' , '2017-05-03' , '2016-04-29' , '2016-05-04'))
df = df.append(get_ga_data('Early May - Monday 1 Day Event' , '2017-05-01' , '2017-05-01' , '2016-05-02' , '2016-05-02'))
df = df.append(get_ga_data('Late May - Pre Event' , '2015-01-01' , '2015-01-01' , '2016-05-20' , '2016-05-25'))
df = df.append(get_ga_data('Late May - Weekend Event' , '2017-05-22' , '2017-05-31' , '2016-05-26' , '2016-06-01'))
df = df.append(get_ga_data('Late May - Monday 1 Day Event' , '2017-05-29' , '2017-05-29' , '2016-05-30' , '2016-05-30'))
df = df.append(get_ga_data('Late May - Price Crash' , '2017-05-04' , '2017-05-04' , '2016-08-04' , '2016-08-04'))

In [8]:
df.to_excel('example.xls')
